#IBM Applied Data Science Capstone Course by Coursera¶

#Week 5 Final Report

###Opening a New bar in Mumbai City, India

* Build a dataframe of neighborhoods in Mumbai,India by web scraping the data from Wikipedia page
* Get the geographical coordinates of the neighborhoods
* Obtain the venue data for the neighborhoods from Foursquare API
* Explore and cluster the neighborhoods
* Select the best cluster to open a new Bar.



In [68]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


In [0]:
# send the GET request
source = requests.get('https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Mumbai').text

In [0]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [0]:
# create a list to store neighborhood data
neighborhoodList = []

In [0]:
# Create list of excluding titles
exclude  = ['Western Suburbs','Eastern Suburbs','Harbour Suburbs','South Mumbai','Other','References']

In [0]:
for row in soup.findAll("span", {"class": "toctext"}):
  if row.get_text() not in exclude:
    neighborhoodList.append(row.get_text())

In [74]:
# create a new DataFrame from the list
mum_df = pd.DataFrame({"Neighborhood": neighborhoodList})

mum_df.head()

,Neighborhood
0,Andheri
1,Bhayandar
2,Bandra
3,Borivali
4,Dahisar


In [75]:
# print the number of rows of the dataframe
mum_df.shape

(39, 1)

In [0]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Mumbai, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [0]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in mum_df["Neighborhood"].tolist() ]

In [78]:
coords

[[19.118459378296492, 72.84176321065843],
 [19.307430000000068, 72.85184000000004],
 [19.054370000000063, 72.84017000000006],
 [19.229360000000042, 72.85751000000005],
 [19.250030000000038, 72.85907000000003],
 [19.164550000000077, 72.84946000000008],
 [19.137920000000065, 72.84941000000003],
 [19.014920000000075, 72.84522000000004],
 [19.207100000000025, 72.83498000000003],
 [19.205760000000055, 72.86953000000005],
 [19.06913000000003, 72.84640000000007],
 [19.186550000000068, 72.84842000000003],
 [19.081770000000063, 72.84205000000003],
 [19.07934000000006, 72.83916000000005],
 [19.01657000000006, 72.85853000000003],
 [19.100630000000024, 72.84378000000004],
 [19.145560000000046, 72.94856000000004],
 [19.08652321008152, 72.90900774216628],
 [19.131380000000036, 72.93568000000005],
 [19.064980000000048, 72.88069000000007],
 [19.171830000000057, 72.95565000000005],
 [19.123100000000022, 72.90942000000007],
 [19.023270726614363, 72.84390704954123],
 [19.111090000000047, 72.9278100000000

In [0]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [0]:
# merge the coordinates into the original dataframe
mum_df['Latitude'] = df_coords['Latitude']
mum_df['Longitude'] = df_coords['Longitude']

In [81]:
print(mum_df.shape)
mum_df

(39, 3)


,Neighborhood,Latitude,Longitude
0,Andheri,19.118459,72.841763
1,Bhayandar,19.307430,72.851840
2,Bandra,19.054370,72.840170
3,Borivali,19.229360,72.857510
4,Dahisar,19.250030,72.859070
5,Goregaon,19.164550,72.849460
6,Jogeshwari,19.137920,72.849410
7,Juhu,19.014920,72.845220
8,Kandivali west,19.207100,72.834980
9,Kandivali east,19.205760,72.869530


In [0]:
# save the DataFrame as CSV file
mum_df.to_csv("mum_df.csv", index=False)


4. Create a map of Mumbai with neighborhoods superimposed on top

In [84]:
address = 'Mumbai, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai, India 18.9387711, 72.8353355.


In [88]:
# create map of Mumbai using latitude and longitude values
map_mum = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(mum_df['Latitude'], mum_df['Longitude'], mum_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_mum)  
    
map_mum

**Use the Foursquare API to explore the neighborhoods**

In [0]:
 # define Foursquare Credentials and Version
CLIENT_ID = 'Your client ID' # your Foursquare ID
CLIENT_SECRET = 'Your client Secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [0]:

radius = 5000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(mum_df['Latitude'], mum_df['Longitude'], mum_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [109]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(3812, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Andheri,19.118459,72.841763,Merwans Cake shop,19.119300,72.845418,Bakery
1,Andheri,19.118459,72.841763,Naturals,19.111204,72.837255,Ice Cream Shop
2,Andheri,19.118459,72.841763,Radha Krishna Veg Restaurant,19.115130,72.843060,Indian Restaurant
3,Andheri,19.118459,72.841763,Joey's Pizza,19.126762,72.830001,Pizza Place
4,Andheri,19.118459,72.841763,PVR Cinemas,19.114637,72.827452,Multiplex


Let's check how many venues were returned for each Neighborhood

In [110]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Andheri,100,100,100,100,100,100
Antop Hill,100,100,100,100,100,100
Bandra,100,100,100,100,100,100
Bhandup,100,100,100,100,100,100
Bhayandar,34,34,34,34,34,34
Borivali,100,100,100,100,100,100
Byculla,100,100,100,100,100,100
Chembur,100,100,100,100,100,100
Colaba,100,100,100,100,100,100


In [111]:
# Number of Unique Category
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 160 uniques categories.


In [112]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Bakery', 'Ice Cream Shop', 'Indian Restaurant', 'Pizza Place',
       'Multiplex', 'Breakfast Spot', 'Falafel Restaurant', 'Theater',
       'Coffee Shop', 'Sandwich Place', 'American Restaurant',
       'Seafood Restaurant', 'Juice Bar', 'Café', 'Beach', 'Food Truck',
       'Hotel', 'Brewery', 'Mediterranean Restaurant',
       'Chinese Restaurant', 'Cocktail Bar', 'Club House', 'Bar', 'Pub',
       'Mughlai Restaurant', 'Italian Restaurant', 'Lounge', 'BBQ Joint',
       'Snack Place', 'Gym / Fitness Center', 'Dessert Shop',
       'Comfort Food Restaurant', 'Diner', 'Movie Theater',
       'Spanish Restaurant', 'Spa', 'Cupcake Shop', 'Nightclub',
       'South Indian Restaurant', 'Gym', 'Restaurant', 'Sculpture Garden',
       'General College & University', 'Fast Food Restaurant',
       'Recreation Center', 'Mexican Restaurant', 'Train Station',
       'Department Store', 'Convenience Store', 'Garden'], dtype=object)

In [113]:
# check if the results contain "Shopping Mall"
"Bar" in venues_df['VenueCategory'].unique()

True

**Analyze Each Neighborhood**

In [114]:
# one hot encoding
mum_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mum_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [mum_onehot.columns[-1]] + list(mum_onehot.columns[:-1])
mum_onehot = mum_onehot[fixed_columns]

print(mum_onehot.shape)
mum_onehot.head()

(3812, 161)


,Neighborhoods,Afghan Restaurant,Airport Service,American Restaurant,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Big Box Store,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Café,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Auditorium,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,General College & University,General Entertainment,German Restaurant,Goan Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Historic Site,History Museum,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Juice Bar,Light Rail Station,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Other Great Outdoors,Outdoors & Recreation,Park,Parsi Restaurant,Performing Arts Venue,Pizza Place,Planetarium,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Recreation Center,Resort,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track,Train Station,Vegetarian / Vegan Restaurant,Water Park,Wine Bar,Women's Store
0,Andheri,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [115]:
# Group the Neighborhoods
mum_grouped = mum_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(mum_grouped.shape)
mum_grouped.head()

(39, 161)


,Neighborhoods,Afghan Restaurant,Airport Service,American Restaurant,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Big Box Store,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Café,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Auditorium,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,General College & University,General Entertainment,German Restaurant,Goan Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Historic Site,History Museum,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Juice Bar,Light Rail Station,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Other Great Outdoors,Outdoors & Recreation,Park,Parsi Restaurant,Performing Arts Venue,Pizza Place,Planetarium,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Recreation Center,Resort,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track,Train Station,Vegetarian / Vegan Restaurant,Water Park,Wine Bar,Women's Store
0,Andheri,0.0,0.0,0.02,0.00,0.0,0.00,0.00,0.0,0.01,0.00,0.02,0.04,0.03,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.01,0.01,0.00,0.0,0.070000,0.0,0.0,0.030000,0.00,0.01,0.02,0.040000,0.00,0.0,0.01,0.00,0.000000,0.0,0.00,0.0,0.01,0.00,0.00,0.000000,0.02,0.00,0.01,0.0,0.0,0.00,0.01,0.00,0.000000,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.010000,0.00,0.00,0.0,0.00,0.00,0.000000,0.0,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.020000,0.00,0.0,0.0,0.0,0.00,0.09,0.0,0.050000,0.110000,0.00,0.00,0.01,0.0,0.04,0.0,0.02,0.00,0.0,0.01,0.0,0.000000,0.00,0.0,0.0,0.00,0.01,0.01,0.00,0.030000,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.020000,0.0,0.000000,0.00,0.0,0.05,0.00,0.000000,0.0,0.010000,0.00,0.0,0.020000,0.00,0.000000,0.05,0.000000,0.00,0.0,0.01,0.0,0.01,0.01,0.01,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.01,0.0,0.0,0.0,0.000000,0.00,0.0,0.00,0.0
1,Antop Hill,0.0,0.0,0.00,0.00,0.0,0.01,0.01,0.0,0.00,0.00,0.02,0.04,0.01,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.01,0.00,0.00,0.0,0.060000,0.0,0.0,0.010000,0.00,0.00,0.00,0.050000,0.00,0.0,0.00,0.00,0.000000,0.0,0.00,0.0,0.01,0.00,0.02,0.000000,0.03,0.01,0.00,0.0,0.0,0.00,0.00,0.00,0.020000,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.000000,0.00,0.00,0.0,0.00,0.01,0.010000,0.0,0.000000,0.00,0.01,0.01,0.01,0.02,0.00,0.00,0.000000,0.01,0.0,0.0,0.0,0.00,0.03,0.0,0.050000,0.200000,0.00,0.01,0.02,0.0,0.01,0.0,0.03,0.01,0.0,0.00,0.0,0.000000,0.01,0.0,0.0,0.00,0.00,0.00,0.01,0.010000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.020000,0.0,0.010000,0.00,0.0,0.01,0.01,0.000000,0.0,0.020000,0.00,0.0,0.000000,0.00,0.000000,0.02,0.000000,0.00,0.0,0.05,0.0,0.02,0.00,0.00,0.0,0.0,0.01,0.0,0.0,0.0,0.00,0.02,0.01,0.00,0.0,0.0,0.0,0.000000,0.02,0.0,0.00,0.0
2

In [116]:
len(mum_grouped[mum_grouped["Bar"] > 0])

38

In [0]:
# New Dataframe with bar in neighnorhood
mum_bar = mum_grouped[["Neighborhoods","Bar"]]

In [120]:
mum_bar.head()

,Neighborhoods,Bar
0,Andheri,0.04
1,Antop Hill,0.04
2,Bandra,0.06
3,Bhandup,0.02
4,Bhayandar,0.00


**Cluster Neighborhoods**  
Run k-means to cluster the neighborhoods in Kuala Lumpur into 3 clusters.

In [123]:
kclusters = 3

mum_clustering = mum_bar.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mum_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 0, 2, 2, 2, 1, 0, 0, 2], dtype=int32)

In [0]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
mum_merged = mum_bar.copy()

# add clustering labels
mum_merged["Cluster Labels"] = kmeans.labels_

In [126]:
mum_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
mum_merged.head()

,Neighborhood,Bar,Cluster Labels
0,Andheri,0.04,1
1,Antop Hill,0.04,1
2,Bandra,0.06,0
3,Bhandup,0.02,2
4,Bhayandar,0.00,2


In [128]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
mum_merged = mum_merged.join(mum_df.set_index("Neighborhood"), on="Neighborhood")

print(mum_merged.shape)
mum_merged.head() # check the last columns!

(39, 5)


,Neighborhood,Bar,Cluster Labels,Latitude,Longitude
0,Andheri,0.04,1,19.118459,72.841763
1,Antop Hill,0.04,1,19.026140,72.866450
2,Bandra,0.06,0,19.054370,72.840170
3,Bhandup,0.02,2,19.145560,72.948560
4,Bhayandar,0.00,2,19.307430,72.851840


In [130]:
print(mum_merged.shape)
mum_merged.sort_values(["Cluster Labels"], inplace=True)
mum_merged.head()

(39, 5)


,Neighborhood,Bar,Cluster Labels,Latitude,Longitude
8,Colaba,0.06,0,18.91542,72.82592
34,Vasai,0.07,0,19.07934,72.83916
2,Bandra,0.06,0,19.05437,72.84017
23,Khar,0.07,0,19.06913,72.84640
24,Kurla,0.07,0,19.06498,72.88069


Visualize the result

In [131]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mum_merged['Latitude'], mum_merged['Longitude'], mum_merged['Neighborhood'], mum_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

** 8. Examine Clusters**

Cluster 0

In [133]:
len(mum_merged.loc[mum_merged['Cluster Labels'] == 0])

8

In [134]:
len(mum_merged.loc[mum_merged['Cluster Labels'] == 1])

16

In [135]:
len(mum_merged.loc[mum_merged['Cluster Labels'] == 2])

15

**Observations:**
Most of the Bars are concentrated in the central area of Mumbai city, with the highest number in cluster 2 and in cluster 3. On the other hand, cluster 1 has low to moderate number of Bars in the neighborhoods. This represents a great opportunity and high potential areas to open new Bar as there is very little to no competition from existing Bars. Owners are advised to avoid neighborhoods in cluster 2 and 3 which already have high concentration of Bars and suffering from intense competition.